In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import re
import time
import os

import sys
sys.path.append("..")

from configs.bad_direction_kpi_dict import bad_direction_kpi_dict as dir_dict
from configs.kpi_constraints_dict import kpi_constraints_dict
from configs.tests2 import drop_missing_data, test_stationarity, test_seasonality, ACF_condition
from configs.tests2 import test_seasonality_vectorized, pivot_maker_vectorized, seas_testing_vectorized

from collections import OrderedDict
# kpi_constraints_dict for is_it_constant

from statsmodels.tsa.stattools import acf
from datetime import datetime
import datetime

#pd.set_option('display.max_rows', None)
import dask.dataframe as dd

import matplotlib.pyplot as plt

In [2]:
import dask
from dask.distributed import Client
client = Client( n_workers = 68,  threads_per_worker=2) #scheduler_address=':37243'
client

2022-09-20 08:42:35,408 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-1gos1ezt', purging
2022-09-20 08:42:35,408 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ugk9iy8v', purging
2022-09-20 08:42:35,409 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-529432lr', purging
2022-09-20 08:42:35,409 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-sb8ju64i', purging
2022-09-20 08:42:35,410 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-i3jb5s1z', purging
2022-09-20 08:42:35,410 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-tj_rgqhu', purging
2022-09-20 08:42:35,410 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9qpqg0od', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 68
Total threads: 136,Total memory: 314.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34589,Workers: 68
Dashboard: http://127.0.0.1:8787/status,Total threads: 136
Started: Just now,Total memory: 314.49 GiB
Comm: tcp://127.0.0.1:35015,Total threads: 2
Dashboard: http://127.0.0.1:41132/status,Memory: 4.62 GiB
Nanny: tcp://127.0.0.1:38892,


# Meta-metadata

In [10]:
file = "/bigdisk/elbaanh/AIO-dev/data/4weeks-lte_enodeb_id-1614153600.csv"
MAX_MISSING_RATIO = .35


ts = 1614153600 #! comes from config, right?
day_of_week = pd.to_datetime(ts,unit='s').dayofweek

# TODO ezt meg akarom mutatni Tamásnak
# initial ts: ts of last datapoint - max(df.ts) esentially 
ts = ts + ((3600 * 24) * (6-day_of_week)) # shift forward to next monday
ts = ts + ((24*3600) - ts % (24*3600)) # shift back to 00:00 on that monday
END = pd.to_datetime(ts, unit='s')
LAST_WEEK_START = END-pd.Timedelta('1 w')
START = END-pd.Timedelta('4 w')

DIM_NAME = "lte_enodeb_id" #! debug, ezt elvileg a címből kéne kiolvasni

In [ ]:
def two_weeks_check(timeseries: pd.Series, ts_cols: list) -> bool:
    """Check whether a timeseries consists of longer than
    14 days of valid data.
    
    The row is stripped off of its metadata, the missing
    data is masked out. Then check if there is two weeks
    between the start and end of timeseries.

    The reason for this step, is that there has to be 
    at least 2 full weeks of data for the Prophet model 
    to estimate the seasonality correctly.

    Parameters
    ----------
    timeseries: pd.Series
        Row of dataframe, the timeseries we want to check
    ts_cols: list
        list of pd._libs.tslibs.timestamps.Timestamp,
        timestamps held in the original dataframe
        

    Returns
    -------
    bool 
        True if the timeseries data is longer than 2 weeks 
        (14+ days), false otherwise.
    """

    interesting_timeseries = timeseries[ts_cols]
    mask = interesting_timeseries.notna()
    interesting_part = interesting_timeseries[mask]
    days_between = interesting_part.index.max() - interesting_part.index.min()
    return days_between.days > 14

# Read dataframe, data cleaning

In [11]:
raw_df = pd.read_csv(file, sep=",", index_col=0)

In [12]:
def clean(df: pd.DataFrame):
    valid_dim_values_mask = df[DIM_NAME].notnull()
    df = df[valid_dim_values_mask]  # remove rows with missing DIM_VALUES

    # basic statistics for every KPI or ratio
    description = df.describe(datetime_is_numeric=True).T

    useful_col_mask = (description["std"] != 0) & (description["count"] > 1.0)
    cols_to_use = list(description[useful_col_mask].index)

    if DIM_NAME not in cols_to_use:
        cols_to_use.append(DIM_NAME)
            
    return df[cols_to_use]


# Here we get rid of KPIs/ratios that are not interesting:
# either the data is not enough
# or there is no variance in the data
main_df = clean(raw_df)
main_df['dt'] = pd.to_datetime(main_df.ts, unit='s')
DIM_VALUES = main_df[DIM_NAME].unique()

In [13]:
# Slice of main_df: one dimension-value pair
dim_value = 80038.0

def preproc_by_dim_value(dim_value: object):
    """
    Mask out a slice of the `main_df` with given `dim_value`,
    clean and impute missing timestamps between START and END. 
    Datapoints outside the START-END interval are abandoned.


    """

    df_dim_raw = main_df[ main_df[DIM_NAME] ==  dim_value ] #TODO debug: remove_trailing _raw
    df_dim = clean(df_dim_raw) #! same as above

    # generate timestamps with uniform steps - some data might be missing
    time_slice = pd.date_range(START, END, freq="H")
        #pd.date_range(df_dim["dt"].min(), df_dim["dt"].max(), freq='H')
    df_dim = df_dim.drop_duplicates(subset = ["dt", DIM_NAME])
    df_dim = df_dim.set_index(["dt", DIM_NAME])

    # reindexing with imputed timestamps
    new_multi_index = pd.MultiIndex.from_product([time_slice, df_dim.index.levels[1]],
                                                    names = ["dt", DIM_NAME])
    df_dim = df_dim.reindex( new_multi_index )
    df_dim = df_dim.reset_index()
    return df_dim

# Process ALL KPI - One DIM

In [75]:
df_pivot.iloc[:,:-6]

,dt,2021-02-01 00:00:00,2021-02-01 01:00:00,2021-02-01 02:00:00,2021-02-01 03:00:00,2021-02-01 04:00:00,2021-02-01 05:00:00,2021-02-01 06:00:00,2021-02-01 07:00:00,2021-02-01 08:00:00,2021-02-01 09:00:00,...,2021-02-28 17:00:00,2021-02-28 18:00:00,2021-02-28 19:00:00,2021-02-28 20:00:00,2021-02-28 21:00:00,2021-02-28 22:00:00,2021-02-28 23:00:00,2021-03-01 00:00:00,missing_data_ratio_all,missing_data_ratio_last_week
variable,lte_enodeb_id,,,,,,,,,,,,,,,,,,,,,
abnormal_lte_conn_rel_ratio_sample,80038.0,2.339200e+04,2.627700e+04,2.405500e+04,1.530500e+04,1.721300e+04,1.777200e+04,1.586200e+04,1.441700e+04,1.218500e+04,1.206700e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177083,0.684524
abnormal_lte_conn_rel_ratio_sum,80038.0,2.338600e+04,2.627700e+04,2.404700e+04,1.529100e+04,1.721000e+04,1.776400e+04,1.585200e+04,1.441600e+04,1.181900e+04,1.206700e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177083,0.684524
abnormal_lte_conn_rel_ratio_wmean,80038.0,9.997435e-01,1.000000e+00,9.996674e-01,9.990853e-01,9.998257e-01,9.995499e-01,9.993696e-01,9.999306e-01,9.699631e-01,1.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177083,0.684524
bytes_dl_sum,80038.0,2.757442e+10,3.567175e+10,2.952289e+10,1.781515e+10,2.373495e+10,2.102825e+10,2.130091e+10,1.927224e+10,1.619879e+10,1.472600e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.174107,0.684524
ims_abnorm_sess_rel_ratio_sample,80038.0,1.630000e+02,1.270000e+02,1.370000e+02,1.050000e+02,8.000000e+01,5.600000e+01,5.300000e+01,2.900000e+01,5.100000e+01,3.500000e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177083,0.684524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
x2_ho_exec_time_sum,80038.0,3.690470e+02,3.363770e+02,3.207925e+02,1.738750e+02,1.883470e+02,1.374265e+02,9.482500e+01,6.157200e+01,4.911200e+01,3.351400e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177083,0.684524
x2_ho_exec_time_wmean,80038.0,1.399496e-01,1.422915e-01,1.392933e-01,1.261792e-01,1.356967e-01,1.370155e-01,1.283153e-01,1.197899e-01,1.200782e-01,1.227619e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177083,0.684524
x2_ho_sample,80038.0,3.091000e+03,2.834000e+03,2.762000e+03,1.609000e+03,1.610000e+03,1.193000e+03,8.840000e+02,6.110000e+02,4.760000e+02,3.370000e+02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175595,0.684524


In [74]:
# Notes:
# in df_pivot each row represents a timeseries,
# with 
# x.name[0]  is KPI/ratio name
# 
# axis=1 means that each operation is done rowwise

df_dim = preproc_by_dim_value(dim_value)
if df_dim.empty:
    pass # TODO return df_dim

df_melt = df_dim.melt(id_vars = [DIM_NAME, "dt" ], 
                      value_vars = df_dim.columns.drop([DIM_NAME, "ts", "dt"]))
df_pivot = df_melt.pivot(index = ["variable",DIM_NAME], 
                             columns = "dt", 
                             values ="value")

# Collect timestamp type columns
ts_cols_fun = lambda col: type(col) == pd._libs.tslibs.timestamps.Timestamp
ts_cols = [*filter(ts_cols_fun, df_pivot.columns)]

# Add misc metadata

# Identify missing values
df_pivot["missing_data_ratio_all"] = df_pivot.apply(
    lambda x: 1 - (sum(~x.isna()) / (4 * 7 * 24)), axis=1
)
df_pivot["missing_data_ratio_last_week"] = df_pivot.apply(
    lambda x: 1 - (sum(~x[LAST_WEEK_START:].isna()) / (7 * 24)), axis=1
)
df_pivot["path"] = file
df_pivot["ts"] = ts
df_pivot["table"] = DIM_NAME + str(ts)
df_pivot["dimension_name"] = df_pivot.apply(
    lambda x: OrderedDict([(DIM_NAME, x.name[1])]), axis=1
)
df_pivot["kpi_name"] = df_pivot.apply(lambda x: x.name[0], axis=1)

# Weed out constant and bad quality timeseries
df_pivot["model_type"] = df_pivot.apply(
    lambda x: "bad_quality_data"
    if x["missing_data_ratio_all"] > MAX_MISSING_RATIO
    else None,
    axis=1
)
    
df_pivot["model_type"] = df_pivot.apply(
    lambda x: "constant" 
    if ((x[:-6].quantile(0.51)==kpi_constraints_dict[ x.name[0] ]) 
        or (x[:-6].quantile(0.49)==kpi_constraints_dict[ x.name[0] ]) 
        or (x[:-6].std() < 10**(-14))) 
    & (x.model_type!="bad_quality_data") 
    else x.model_type, 
    axis=1
)
    # Check if timeseries is 14+ days
df_pivot["model_type"] = df_pivot.apply(
    lambda x: "bad_quality_data" 
    if not two_weeks_check(x, ts_cols) 
        & (x.model_type not in ("bad_quality_data", "constant"))
    else x.model_type,
    axis=1
)

bad_quality_mask = df_pivot["model_type"].isin(["constant","bad_quality_data"])
df_bad_quality = df_pivot[bad_quality_mask]

df_bad_quality[
    [
        "seasonality_flag",
        "statonarity_flag",
        "missing_data_imputation_flag",
        "nan_trimming_flag",
    ]
] = (None, None, None, None)

# Good quality data, 
df2 = df_pivot[~bad_quality_mask]

/tmp/ipykernel_11400/3693399139.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[
/tmp/ipykernel_11400/3693399139.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[
/tmp/ipykernel_11400/3693399139.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [70]:
df2

NameError: name 'df2' is not defined

# One DIM